In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape
from model import CNN_with_mask
from get_training_data import get_rally_result, get_padding_data, permute_feature

from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1

np.random.seed(44)
tf.random.set_seed(44)

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [2]:
df = pd.read_csv('new_test.csv')

In [3]:
# feature_list = [['Player', 'Space'], ['Player', 'Action'], ['Player', 'Sequence'], ['Space', 'Action'], ['Space', 'Sequence'], ['Action', 'Sequence']]
feature_list = [['Player', 'Space', 'Sequence'], ['Space', 'Action', 'Sequence'], ['Player', 'Action', 'Sequence'], ['Player', 'Space', 'Action']]

In [4]:
def create_model(rally_size, shot_size, feature_dim, space_embed_size, action_embed_size, shot_embed_size):
    '''
    framework: 
    1. 對 space, action 做 embeding, (input, output) = (feature_dim, embed_size)
    2. concat space, action, others 成一個 embedded vector for each atk, (input) =  ([feature_dim, embed_size, embed_size])
    3. 先做 embedding
    4. CNN, filters = shot_embed_size
    5. GRU
    '''
    # each input: 三個維度, rally shot feature
    input_others = keras.Input(shape=(rally_size, shot_size, feature_dim[0]))
    input_space = keras.Input(shape=(rally_size, shot_size))
    input_action = keras.Input(shape=(rally_size, shot_size))

    # space & action 先做 embedding, 再和 others concat
    embed_space_layer = Embedding(input_dim=feature_dim[1], output_dim=space_embed_size, mask_zero=True, name='Space_Embedding')
    embed_action_layer = Embedding(input_dim=feature_dim[2], output_dim=action_embed_size, mask_zero=True, name='Action_Embedding')
    masking_layer = Masking(mask_value=0)
    concat_layer = Concatenate(name='Input_Concat')

    embed_shot_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=3, name='Shot_Embedding')
    cnn_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=1, name='CNN_Layer')
    gru_layer = GRU(units=16, name='GRU_Layer')

    dense_layer = Dense(units=2, activation='softmax')

    # forward
    inputs = [input_others, input_space, input_action]
    embed_space = embed_space_layer(input_space)
    embed_action = embed_action_layer(input_action)
    masked_others = masking_layer(tf.cast(input_others, tf.float32))
    embed_input = concat_layer([masked_others, embed_space, embed_action])
    embed_shot = tf.squeeze(embed_shot_layer(embed_input), axis=2)
    cnn_output = cnn_layer(embed_shot)
    gru_output = gru_layer(cnn_output)
    output = dense_layer(gru_output)
    model = keras.Model(inputs=inputs, outputs=output, name='Classification')

    return model

In [5]:
def get_data(df):
    space_replace = {list(df.groupby('Space').groups.keys())[i]: i+1 for i in range(len(df.groupby('Space')))}
    action_replace = {list(df.groupby('Action').groups.keys())[i]: i+1 for i in range(len(df.groupby('Action')))}
    df = pd.get_dummies(df, columns=['Player'])
    df = df.replace(space_replace)
    df = df.replace(action_replace)
    # get label
    label, ignor = get_rally_result(df)
    # get training data
    rally_set, rally_space_set, rally_action_set = get_padding_data(df, ignor)
    rally_space_set = rally_space_set.squeeze()
    rally_action_set = rally_action_set.squeeze()

    return rally_set, rally_space_set, rally_action_set, label

def data_transfer_to_tensor(rally_set, rally_space_set, rally_action_set, label):
    rally_set_tensor = tf.convert_to_tensor(rally_set)
    rally_space_set_tensor = tf.convert_to_tensor(rally_space_set)
    rally_action_set_tensor = tf.convert_to_tensor(rally_action_set)
    rally_result_tensor = tf.convert_to_tensor(label)
    
    return rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor, rally_result_tensor

def split_data(others_tensor, space_tensor, action_tensor, label_tensor):
    l = label_tensor.shape[0]
    split_persentage = int(l*0.9)

    train_space = space_tensor[:split_persentage]
    train_action = action_tensor[:split_persentage]
    train_others = others_tensor[:split_persentage]
    train_label = label_tensor[:split_persentage]

    test_space = space_tensor[split_persentage:]
    test_action = action_tensor[split_persentage:]
    test_others = others_tensor[split_persentage:]
    test_label = label_tensor[split_persentage:]

    train_x = [train_others, train_space, train_action]
    train_y = train_label

    test_x = [test_others, test_space, test_action]
    test_y = test_label

    return train_x, train_y, test_x, test_y

In [6]:
def training(model, train_x, train_y):
    optimizer = 'adam'
    loss = keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    epochs = 30
    callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True, monitor='val_loss')

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.fit(train_x, train_y, epochs=epochs, validation_split=0.1, callbacks=[callbacks])

    return model

In [7]:
def output_result(type, feature_list, score):
    if(type == 'permute'):
        output_path = 'permute_feature_result.txt'
        f = open(output_path, 'a')
        f.write('Permuted feature: ' + str(feature_list) + '\n')
        f.write('-'*40 + '\n')
        f.write('accuracy:    {:.2f}'.format(score[0]) + '\n')
        f.write('f1:          {:.2f}'.format(score[1]) + '\n')
        f.write('auc:         {:.2f}'.format(score[2]) + '\n')
        f.write('BS:          {:.2f}'.format(score[3]) + '\n')
        f.write('\n' + '='*40 + '\n')
    elif(type == 'mode'):
        output_path = 'replaced_by_mode_result.txt'
        f = open(output_path, 'a')
        f.write('Fixed feature: ' + str(feature_list) + '\n')
        f.write('-'*40 + '\n')
        f.write('accuracy:    {:.2f}'.format(score[0]) + '\n')
        f.write('f1:          {:.2f}'.format(score[1]) + '\n')
        f.write('auc:         {:.2f}'.format(score[2]) + '\n')
        f.write('BS:          {:.2f}'.format(score[3]) + '\n')
        f.write('\n' + '='*40 + '\n')
    f.close()

In [8]:
# feature_list = [['Player', 'Space'], ['Player', 'Action'], ['Player', 'Sequence'], ['Space', 'Action'], ['Space', 'Sequence'], ['Action', 'Sequence']]
# feature_list = [['Player', 'Space', 'Sequence'], ['Space', 'Action', 'Sequence'], ['Player', 'Action', 'Sequence'], ['Player', 'Space', 'Action']]

In [9]:
def replaced_by_mode(df, ex_feature):
    if(ex_feature == 'Original'):
        return df
    col = ['Player', 'Space', 'Action']
    col.remove(ex_feature)
    for c in col:
        mode = df.mode(axis=0)[c][0]
        replace = {list(df.groupby(c).groups.keys())[i]: mode for i in range(len(df.groupby(c)))}
        df[c] = df[c].replace(replace)

    return df

In [10]:
experiment = ['Action']

############# permutation #############################
for f in experiment:
    # df = permute_feature(df, f)
    df = replaced_by_mode(df, f)
############# train for result ########################
rally_set, rally_space_set, rally_action_set, label = get_data(df)
rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor, rally_result_tensor = data_transfer_to_tensor(rally_set, rally_space_set, rally_action_set, label)
train_x, train_y, test_x, test_y = split_data(rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor, rally_result_tensor)
############# model setting ###########################
rally_size = rally_set.shape[1]
shot_size = 3
feature_dim = (rally_set.shape[-1], len(df.groupby('Space'))+1, len(df.groupby('Action'))+1)
space_embed_size = 8
action_embed_size = 8
shot_embed_size = 16
############# create model ############################
model = create_model(rally_size, shot_size, feature_dim, space_embed_size, action_embed_size, shot_embed_size)
model = training(model, train_x, train_y)
############# evaluate ################################
y_pred = model.predict(test_x)
argmax_y_pred = np.argmax(y_pred, axis=1)
argmax_test_y = np.argmax(test_y, axis=1)

acc_score = accuracy_score(argmax_test_y, argmax_y_pred)
f1_score = f1(argmax_test_y, argmax_y_pred)
auc_score = roc_auc_score(test_y, y_pred)
BS = calculate_BS(test_y, y_pred, 2)
score = [acc_score, f1_score, auc_score, BS['BS'][0]]
# output_result('permute', experiment, score)
output_result('mode', experiment, score)

Epoch 1/30
33/33 [==============================] - 2s 16ms/step - loss: 0.6554 - accuracy: 0.6609 - val_loss: 0.6039 - val_accuracy: 0.7094
Epoch 2/30
33/33 [==============================] - 0s 7ms/step - loss: 0.6430 - accuracy: 0.6600 - val_loss: 0.6071 - val_accuracy: 0.7094
Epoch 3/30
33/33 [==============================] - 0s 6ms/step - loss: 0.6431 - accuracy: 0.6600 - val_loss: 0.6075 - val_accuracy: 0.7094
Epoch 4/30
33/33 [==============================] - 0s 6ms/step - loss: 0.6430 - accuracy: 0.6600 - val_loss: 0.6172 - val_accuracy: 0.7094
Epoch 5/30
33/33 [==============================] - 0s 6ms/step - loss: 0.6421 - accuracy: 0.6609 - val_loss: 0.6078 - val_accuracy: 0.7094
Epoch 6/30
33/33 [==============================] - 0s 6ms/step - loss: 0.6409 - accuracy: 0.6609 - val_loss: 0.6081 - val_accuracy: 0.7094
Epoch 7/30
33/33 [==============================] - 0s 6ms/step - loss: 0.6399 - accuracy: 0.6609 - val_loss: 0.6094 - val_accuracy: 0.7094
Epoch 8/30
33/33 [=